# Read Data in the Database

Before loading the data present in the database, it is important to explain how this data is organized and how the model of the building was taken.

- Demand Floor: This column represents the floor from which the request was made. For this first approximation, it was decided to model a building with 8 floors (-2 to 5), where "-2" and "-1" are the parking floors, "0 " the lobby, and from "1" to "5" the floors of the residents. 

- Temperature : It is the temperature in degrees centigrade of the environment.

- Weather: It is the description of the climate of the environment, for this example 4 possible climates were defined, Sunny, Humid, Raining and Windy. 

- Elevator_weight: This variable indicates the weight that the elevator carries inside, this can help us identify if the elevator was in motion or not when the request was made, this can be useful since if a request was made while it was completing another request, it would not it is necessary to apply the model to find the Resting floor.

- Parking Capacity(6): In the database there are 6 columns that refer to the percentage of occupancy of the parking lots for visitors and for residents, each column represents the parking spaces per floor, in this way if the "Capacity_visitor_parking(%)" column has values ​​in 100 means all visitor parking lots are full.

- Holiday day: This column will help us identify if a day was a holiday or not, it is important since the behavior of the elevator will not be the same on normal days as on holidays, and in this way, regardless of the dates, the model will be able to predict better. Where do he must go

- Demand timestamp: This time stamp will give us the exact date and time a request was made, by default each time a new data is added to the database, a time stamp of the current time will be created, as there can be no 2 equal timestamps, this column is the unique ID of each demand.

## Business rules

- Due to the way in which the data was conceived, there is a high probability of having the same prediction for 30 minutes, this was done to prevent data from being available for all the minutes of the day, so each time was divided into 2 blocks of 30 minutes

- Due to the organization of the data, the prediction of the rest floor of the lowest floor will be the same, so there will be no difference between the prediction of floor "-1" and "-2"

## Code

In [67]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [68]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

db="building_2";
pw="Sebasman12";

connection = create_db_connection("localhost", "root", pw, db) # Connect to the Database

MySQL Database connection successful


In [3]:
q1 = """
SELECT *
FROM Elevator_data;
"""

In [72]:
results = read_query(connection, q1)
from_db = []

for result in results:
  result = list(result)
  from_db.append(result)


#print(from_db)
columns = ["Demand_Floor", "Temperature(C°)", "Weather", "Elevator_weight(Kg)", "Capacity_visitor_parking(%)" , "Capacity_resident_parking_F1(%)" , "Capacity_resident_parking_F2(%)" , "Capacity_resident_parking_F3(%)" , "Capacity_resident_parking_F4(%)" , "Capacity_resident_parking_F5(%)" , "Holiday_day" , "Demand_timestamp"]
#print(columns)
df = pd.DataFrame(from_db, columns=columns)

## DATA IN DATABASE

In [73]:
df_clean = df.copy()
df_clean

,Demand_Floor,Temperature(C°),Weather,Elevator_weight(Kg),Capacity_visitor_parking(%),Capacity_resident_parking_F1(%),Capacity_resident_parking_F2(%),Capacity_resident_parking_F3(%),Capacity_resident_parking_F4(%),Capacity_resident_parking_F5(%),Holiday_day,Demand_timestamp
0,1,16,Sunny,0,27,89,34,67,34,46,YES,2019-01-18 09:12:01
1,2,12,Humid,200,96,0,87,0,46,0,NO,2019-03-02 11:57:01
2,0,8,Raining,0,98,76,40,98,56,87,NO,2019-12-16 10:45:01
3,3,8,Raining,0,2,34,87,99,76,34,NO,2021-05-30 03:28:01
4,5,20,Sunny,0,80,100,100,75,100,50,YES,2022-07-21 15:54:01
5,0,8,Windy,70,100,80,76,56,80,20,NO,2022-10-18 11:55:37
6,0,17,Sunny,70,56,2,80,45,20,65,YES,2022-10-18 11:55:39
7,0,14,Humid,70,100,80,76,56,20,80,NO,2022-10-18 11:55:41


## PREPARE DATA TO INTRODUCE IN A ML MODEL

### Replacing string to numeric

In [74]:
## Sunny   - 0
## Humid   - 1
## Raining - 2
## Windy   - 3
df_clean["Weather"]=df["Weather"].replace(["Sunny","Humid","Raining","Windy"],[0,1,2,3])

## YES - 0
## NO  - 1

df_clean["Holiday_day"]=df["Holiday_day"].replace(["YES","NO"],[0,1])

### Analyze timestamp

In [75]:
df_clean=df_clean.drop(['Demand_timestamp'], axis=1)
df_clean['Year']   = df['Demand_timestamp'].dt.year
df_clean['Month']  = df['Demand_timestamp'].dt.month
df_clean['Day']    = df['Demand_timestamp'].dt.day
df_clean['Hour']   = df['Demand_timestamp'].dt.hour
df_clean['Minute'] = df['Demand_timestamp'].dt.minute
df_clean

,Demand_Floor,Temperature(C°),Weather,Elevator_weight(Kg),Capacity_visitor_parking(%),Capacity_resident_parking_F1(%),Capacity_resident_parking_F2(%),Capacity_resident_parking_F3(%),Capacity_resident_parking_F4(%),Capacity_resident_parking_F5(%),Holiday_day,Year,Month,Day,Hour,Minute
0,1,16,0,0,27,89,34,67,34,46,0,2019,1,18,9,12
1,2,12,1,200,96,0,87,0,46,0,1,2019,3,2,11,57
2,0,8,2,0,98,76,40,98,56,87,1,2019,12,16,10,45
3,3,8,2,0,2,34,87,99,76,34,1,2021,5,30,3,28
4,5,20,0,0,80,100,100,75,100,50,0,2022,7,21,15,54
5,0,8,3,70,100,80,76,56,80,20,1,2022,10,18,11,55
6,0,17,0,70,56,2,80,45,20,65,0,2022,10,18,11,55
7,0,14,1,70,100,80,76,56,20,80,1,2022,10,18,11,55


Because it is likely that data will not be available for all the minutes of all the hours of the day, the minutes will be divided into 2, the first 30 minutes and the second 30 minutes.

In [76]:
## 0-29 minutes / 0 
## 30-59 minutes / 1

In [77]:
df_clean['Minute'][df_clean['Minute'] <30] = 0 
df_clean['Minute'][df_clean['Minute'] >29] = 1
df_clean

,Demand_Floor,Temperature(C°),Weather,Elevator_weight(Kg),Capacity_visitor_parking(%),Capacity_resident_parking_F1(%),Capacity_resident_parking_F2(%),Capacity_resident_parking_F3(%),Capacity_resident_parking_F4(%),Capacity_resident_parking_F5(%),Holiday_day,Year,Month,Day,Hour,Minute
0,1,16,0,0,27,89,34,67,34,46,0,2019,1,18,9,0
1,2,12,1,200,96,0,87,0,46,0,1,2019,3,2,11,1
2,0,8,2,0,98,76,40,98,56,87,1,2019,12,16,10,1
3,3,8,2,0,2,34,87,99,76,34,1,2021,5,30,3,0
4,5,20,0,0,80,100,100,75,100,50,0,2022,7,21,15,1
5,0,8,3,70,100,80,76,56,80,20,1,2022,10,18,11,1
6,0,17,0,70,56,2,80,45,20,65,0,2022,10,18,11,1
7,0,14,1,70,100,80,76,56,20,80,1,2022,10,18,11,1


### Analyze Demand_Floor

In [78]:
def resting_floor (x): 
    if x<-1:
        return x
    else : 
        return x-1 

df_clean['Demand_Floor']=df_clean['Demand_Floor'].map(resting_floor)
df_clean.rename({'Demand_Floor': 'Resting_Floor'}, axis=1)

,Resting_Floor,Temperature(C°),Weather,Elevator_weight(Kg),Capacity_visitor_parking(%),Capacity_resident_parking_F1(%),Capacity_resident_parking_F2(%),Capacity_resident_parking_F3(%),Capacity_resident_parking_F4(%),Capacity_resident_parking_F5(%),Holiday_day,Year,Month,Day,Hour,Minute
0,0,16,0,0,27,89,34,67,34,46,0,2019,1,18,9,0
1,1,12,1,200,96,0,87,0,46,0,1,2019,3,2,11,1
2,-1,8,2,0,98,76,40,98,56,87,1,2019,12,16,10,1
3,2,8,2,0,2,34,87,99,76,34,1,2021,5,30,3,0
4,4,20,0,0,80,100,100,75,100,50,0,2022,7,21,15,1
5,-1,8,3,70,100,80,76,56,80,20,1,2022,10,18,11,1
6,-1,17,0,70,56,2,80,45,20,65,0,2022,10,18,11,1
7,-1,14,1,70,100,80,76,56,20,80,1,2022,10,18,11,1


### Prepare Data to Train ML Model

Separate the Data 

In [63]:
X_complete  = df_clean.iloc[:,1:].values
y_complete  = df_clean.iloc[:,0].values
X_train, X_test, y_train, y_test = train_test_split(X_complete, y_complete, test_size=0.25, random_state=178)

Normalization

In [65]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)


By having this data, the model can be trained in a simple way, where the model will predict where it should go after completing the demand.